The feedback about movies falls into one of two categories:

- `Explicit`— users specify how much they liked a particular movie by providing a numerical rating.
- `Implicit`— if a user watches a movie, the system infers that the user is interested.

he data contain user_id, item_id, and interaction (0-non-interact, 1 – has interact). 

In [27]:
%autosave 100

Autosaving every 100 seconds


# import 

In [25]:
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
from urllib.request import urlretrieve
import zipfile
import os
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
import warnings
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import random
import time
import scipy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.initializers import RandomUniform, he_normal,he_uniform
import math
warnings.filterwarnings('ignore')
%matplotlib inline

## read original data

In [2]:
def load_movielens_data(data_directory):
    ratings_df = pd.read_csv(
    os.path.join(data_directory, "ratings.dat"),
    sep="::",
    names=["UserID", "MovieID", "Rating", "Timestamp"], engine="python")
    movies_df = pd.read_csv(
      os.path.join(data_directory, "movies.dat"),
      sep="::",
      names=["MovieID", "Title", "Genres"], engine="python")

  # Create dictionaries mapping from old IDs to new (remapped) IDs for both
  # MovieID and UserID. Use the movies and users present in ratings_df to
  # determine the mapping, since movies and users without ratings are unneeded.
    movie_mapping = {
      old_movie: new_movie for new_movie, old_movie in enumerate(
          ratings_df.MovieID.astype("category").cat.categories)
  }
    user_mapping = {
      old_user: new_user for new_user, old_user in enumerate(
          ratings_df.UserID.astype("category").cat.categories)
  }

  # Map each DataFrame consistently using the now-fixed mapping.
    ratings_df.MovieID = ratings_df.MovieID.map(movie_mapping)
    ratings_df.UserID = ratings_df.UserID.map(user_mapping)
    movies_df.MovieID = movies_df.MovieID.map(movie_mapping)

  # Remove nulls resulting from some movies being in movies_df but not
  # ratings_df.
    movies_df = movies_df[pd.notnull(movies_df.MovieID)]

    return ratings_df, movies_df

In [3]:
data_directory = '/data/jyu29/data/recommendation_data'
ratings_df, movies_df = load_movielens_data(data_directory)
ratings_df.sample(2)

,UserID,MovieID,Rating,Timestamp
101512,675,344,2,975685011
220535,1335,689,4,974946138


In [14]:
# scipy.sparse.coo_matrix((ratings_df.rating, (ratings_df.userId, ratings_df.movieId)))
ratings_df = ratings_df.rename(columns={"UserID":'userId','MovieID':'movieId','Rating':'rating'})

## negative sampleing

metimes, we want to reduce the training time by using a subset of a very large dataset while the negative samples outnumbers the positive ones, e.g. word embedding. Another situation when we deal with implicit data. In this case, we may need to populate new data for negative values. This post demonstrates how to generate data for training using uniform negative sampling.

If we consider interaction has value 1 and 0 otherwise, then the original rating data will become all 1s. So, you can see that with only 1s in the label, the model cannot distinguish between interact and not interact as shown in the following tables.

scipy.sparse.coo_matrix((ratings_df.rating, (ratings_df.userId, ratings_df.movieId)))We want to get n negative samples per one positive. The naive method can be:

Loop through all user ids
For each user id, get a random item id and check if the pair user-item does not exist in the dataset
Add the found user-item as a negative sample to the dataset.
If you follow these steps, then you may find the execution times can be really long (~20 mins). To accelerate the result, we utilize some useful libraries as follows:

Generate a dense matrix from the dataset using scipy,
rows and cols are users and items
For each row, extract krandom items’ indices of 0 values using random.samples
k is the number of non zero values in that row.
Append the list of user-item from k extract index and append to the dataset.

In [21]:
"""version 1.2: 1 positive 1 neg (2 times bigger than the original dataset by default)
    Parameters:
    input rating data as pandas dataframe: userId|movieId|rating
    n_neg: take n_negative / 1 positive
    Returns:
    negative sampled set as pandas dataframe
            userId|movieId|interact (implicit)
"""


def neg_sampling(ratings_df, n_neg=1, neg_val=0, pos_val=1, percent_print=5):
    ratings_df.userId = ratings_df.userId.astype('category').cat.codes.values
    ratings_df.movieId = ratings_df.movieId.astype('category').cat.codes.values
    sparse_mat = scipy.sparse.coo_matrix((ratings_df.rating, (ratings_df.userId, ratings_df.movieId)))
    dense_mat = np.asarray(sparse_mat.todense())
    print(dense_mat.shape)
    
    nsamples = ratings_df[['userId', 'movieId']]
    nsamples['interact'] = nsamples.apply(lambda row: 1, axis=1)
    length = dense_mat.shape[0]
    printpc = int(length * percent_print/100)
    nTempData = []
    i = 0
    start_time = time.time()
    stop_time = time.time()
    extra_samples = 0
    for row in dense_mat:
        if(i%printpc==0):
            stop_time = time.time()
#             print("processed ... {0:0.2f}% ...{1:0.2f}secs".format(float(i)*100 / length, stop_time - start_time))
            start_time = stop_time
        n_non_0 = len(np.nonzero(row)[0])
        zero_indices = np.where(row==0)[0]
        
        if(n_non_0 * n_neg + extra_samples > len(zero_indices)):
            print(i, "non 0:", n_non_0,": len ",len(zero_indices))
            neg_indices = zero_indices.tolist()
            extra_samples = n_non_0 * n_neg + extra_samples - len(zero_indices)
        else:
            neg_indices = random.sample(zero_indices.tolist(), n_non_0 * n_neg + extra_samples)
            extra_samples = 0
            nTempData.extend([(uu, ii, rr) for (uu, ii, rr) in zip(np.repeat(i, len(neg_indices))
                    , neg_indices, np.repeat(neg_val, len(neg_indices)))])
    i+=1
    nsamples=nsamples.append(pd.DataFrame(nTempData, columns=["userId","movieId", "interact"]),ignore_index=True)
    return nsamples


In [22]:
implicit_feedback_data = neg_sampling(ratings_df, n_neg=1, neg_val=0, pos_val=1, percent_print=5)

(6040, 3706)
0 non 0: 2314 : len  1392


In [24]:
implicit_feedback_data.sample(5)

,userId,movieId,interact
1225953,0,3014,0
1872976,0,1964,0
499894,3068,1478,1
597005,3625,3521,1
1384244,0,27,0


In [25]:
implicit_feedback_data.to_csv('/data/jyu29/data/recommendation_data/implicit_feedback_data.csv',index=False)

# read explicit feedback data

In [24]:
dataset = pd.read_csv('/data/jyu29/data/recommendation_data/implicit_feedback_data.csv')

In [29]:
dataset = dataset.rename(columns={'userId':'user_id',
                                 'movieId':'item_id'})

In [30]:
dataset.user_id = dataset.user_id.astype('category').cat.codes.values
dataset.item_id = dataset.item_id.astype('category').cat.codes.values
train, test = train_test_split(dataset, test_size=0.2)
n_users, n_movies = len(dataset.user_id.unique()), len(dataset.item_id.unique())
n_latent_factors = 50

# build mlp

In [31]:
#movie input and embedding
embedding_init = 'he_uniform'
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies, n_latent_factors,
                                          embeddings_initializer=embedding_init,
                                          embeddings_regularizer=l2(1e-6),
                                          embeddings_constraint='NonNeg',
                                          name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
#user input and embedding
user_input = keras.layers.Input(shape=[1],name='User')
user_embedding = keras.layers.Embedding(n_users, n_latent_factors,
                                          embeddings_initializer=embedding_init,
                                          embeddings_regularizer=l2(1e-6),
                                          embeddings_constraint='NonNeg',
                                          name='User-Embedding')(user_input)
user_vec = keras.layers.Flatten(name='FlattenUsers')(user_embedding)

In [32]:
concat = keras.layers.concatenate([movie_vec, user_vec])
mlp = concat
for i in range(3,-1,-1):
    if i == 0:
        mlp = Dense(8**i, activation='sigmoid', kernel_initializer='glorot_normal',
                  name="output")(mlp)
    else:
        mlp = Dense(8*2**i, activation='relu', kernel_initializer='he_uniform')(mlp)
        if i > 2:
            mlp = BatchNormalization()(mlp)
            mlp = Dropout(0.2)(mlp)
model = Model(inputs=[user_input, movie_input], outputs=[mlp])
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 Movie-Embedding (Embedding)    (None, 1, 50)        185300      ['Item[0][0]']                   
                                                                                                  
 User-Embedding (Embedding)     (None, 1, 50)        302000      ['User[0][0]']                   
                                                                                            

## data generator

In [33]:
class DataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=16, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.dataframe = dataframe
        self.shuffle = shuffle
        self.indices = dataframe.index
        print(len(self.indices))
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.floor(len(self.dataframe) / self.batch_size)
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        idxs = [i for i in range(index*self.batch_size,(index+1)*self.batch_size)]
        #print(idxs)
        # Find list of IDs
        list_IDs_temp = [self.indices[k] for k in idxs]
        # Generate data
        User = self.dataframe.iloc[list_IDs_temp,[0]].to_numpy().reshape(-1)
        Item = self.dataframe.iloc[list_IDs_temp,[1]].to_numpy().reshape(-1)
        rating = self.dataframe.iloc[list_IDs_temp,[2]].to_numpy().reshape(-1)
        #print("u,i,r:", [User, Item],[y])
        return [User, Item],[rating]
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indices = np.arange(len(self.dataframe))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

## train

In [34]:
# early_stop = EarlyStopping(monitor='val_output_loss', min_delta = 0.0001, patience=10)
traindatagenerator = DataGenerator(train)
val_generator = DataGenerator(test)
# history = model.fit(traindatagenerator, validation_data=val_generator, epochs=200, verbose=2, callbacks=[early_stop])

1599220
399806


In [36]:
history = model.fit(traindatagenerator, validation_data=val_generator, epochs=5)

Epoch 1/5
99951/99951 [==============================] - 426s 4ms/step - loss: 0.0371 - binary_accuracy: 0.9976 - val_loss: 4.3709e-04 - val_binary_accuracy: 1.0000
Epoch 2/5
99951/99951 [==============================] - 431s 4ms/step - loss: 0.0011 - binary_accuracy: 0.9999 - val_loss: 4.4422e-04 - val_binary_accuracy: 1.0000
Epoch 3/5
99951/99951 [==============================] - 440s 4ms/step - loss: 7.6047e-04 - binary_accuracy: 0.9999 - val_loss: 4.8007e-04 - val_binary_accuracy: 1.0000
Epoch 4/5
99951/99951 [==============================] - 415s 4ms/step - loss: 6.0043e-04 - binary_accuracy: 1.0000 - val_loss: 4.9763e-04 - val_binary_accuracy: 1.0000
Epoch 5/5
99951/99951 [==============================] - 410s 4ms/step - loss: 6.4766e-04 - binary_accuracy: 1.0000 - val_loss: 5.2456e-04 - val_binary_accuracy: 1.0000
